# Analyze everything

This notebook demonstrates how to run a complete STRESS analysis in a few lines of code and produce all relevant output graphs.

In [1]:
import napari_stress
import napari
import numpy as np
from napari_stress import reconstruction, approximation, measurements, frame_by_frame, TimelapseConverter
import napari_process_points_and_surfaces as nppas
import vedo

## Config

### Data dimensions
You need to set a few parameters pertaining to your data:

In [2]:
voxel_size_x = 1.0  # microns
voxel_size_y = 1.0  # microns
voxel_size_z = 1.93  # microns
target_voxel_size = 1.0  # microns

### Droplet reconstruction parameters

All of the parameters are explained in more detail [here](point_and_click:droplet_reconstruction).

In [3]:
n_smoothing_iterations = 10
n_points_first_guess = 256
n_tracing_iterations = 1
resampling_length = 1
fit_type = 'fancy'  # can be 'fancy' or 'quick'
edge_type = 'interior'  # can be 'interior' or 'surface'
trace_length = 10
trace_sampling_distance = 0.5

### Stress analysis parameters
All of the used parameters are explained in more detail [here](point_and_click:stress_toolbox).

In [4]:
max_degree = 10  # spherical harmonics degree
n_quadrature_points = 434  # number of quadrature points to measure on
gamma = 6.0  # interfacial tension of droplet

In [5]:
viewer = napari.Viewer(ndisplay=3)

In [6]:
example_data = napari_stress.get_droplet_4d()[0][0]
example_data.shape

(21, 16, 30, 31)

In [7]:
viewer.add_image(example_data)

<Image layer 'example_data' at 0x2c8b2f2f760>

In [ ]:
results_reconstruction = reconstruction.reconstruct_droplet(example_data,
                                                            voxelsize=np.asarray([voxel_size_z, voxel_size_y, voxel_size_x]),
                                                            target_voxelsize=target_voxel_size,
                                                            n_smoothing_iterations=n_smoothing_iterations,
                                                            n_points=n_points_first_guess,
                                                            n_tracing_iterations=n_tracing_iterations,
                                                            resampling_length=resampling_length,
                                                            fit_type=fit_type,
                                                            edge_type=edge_type,
                                                            trace_length=trace_length,
                                                            sampling_distance=trace_sampling_distance
                                                            )

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


Processing vertices...:  50%|███████████▉            | 211/426 [00:00<00:00, 342.93it/s]

In [ ]:
for res in results_reconstruction:
    print(res[1]['name'])
    if res[2] == 'image':
        viewer.add_image(res[0], **res[1])
    elif res[2] == 'points':
        viewer.add_points(res[0], **res[1])
    elif res[2] == 'vectors':
        viewer.add_vectors(res[0], **res[1])
refined_points = results_reconstruction[3]

We need the refined points to continue:

In [ ]:
results_stress_analysis = measurements.comprehensive_analysis(refined_points[0],
                                                              max_degree=max_degree,
                                                              n_quadrature_points=n_quadrature_points,
                                                              gamma=gamma)
for res in results_stress_analysis:
    print(res[1]['name'])
    if res[2] == 'points':
        viewer.add_points(res[0], **res[1])
    if res[2] == 'vectors':
        viewer.add_vectors(res[0], **res[1])

Reconstruct the surface: